In [ ]:
# -*- coding: utf-8
# Reinaldo Chaves (reinaldo@abraji.org.br)
'''
Para chegar aos processos de outorga de empreendimentos eólicos no Brasil, realizamos um cadastro
no sistema de consulta processual da Aneel (https://antigo.aneel.gov.br/web/guest/processo-eletronico/-/asset_publisher/a8IsKhUJqQ2q/content/consulta-processual/654778?inheritRedirect=false&redirect=https%3A%2F%2Fantigo.aneel.gov.br%2Fweb%2Fguest%2Fprocesso-eletronico%3Fp_p_id%3D101_INSTANCE_a8IsKhUJqQ2q%26p_p_lifecycle%3D0%26p_p_state%3Dnormal%26p_p_mode%3Dview%26p_p_col_id%3Dcolumn-1%26p_p_col_count%3D1),
que permite aos cidadãos acompanhar o trâmite dos processos administrativos ostensivos da agência. Em seguida, utilizamos a página de dados abertos (https://dadosabertos.aneel.gov.br/dataset/atos-de-outorgas-de-geracao)
da entidade para obter o CSV que contém a lista com todos os documentos emitidos pela Aneel, referentes aos empreendimentos de geração de energia,
a partir de 2015. Filtramos a tabela por SigTipoGeracao, selecionando “EOL - Central Geradora Eólica”, e DscFonteCombustivel, limitando a “cinética do vento”.
A partir daí, utilizamos a Base de Informações sobre os Povos Indígenas e Quilombolas do Instituto Brasileiro de Geografia e Estatística (https://www.ibge.gov.br/geociencias/organizacao-do-territorio/tipologias-do-territorio/27480-base-de-informacoes-sobre-os-povos-indigenas-e-quilombolas.html?edicao=27481&t=acesso-ao-produto)
para identificar os municípios onde há presença dessas duas populações e onde também há um parque eólico autorizado.
Para tanto, foi feito o cruzamento pelo nome das cidades utilizando as colunas 'NomMunicipio', da base da Aneel, e 'NM_MUN', da base do IBGE.
Para evitar duplicidade e falsos positivos, já que há cidades homônimas no país, os nomes dos municípios foram padronizados com retirada de acentos e uso de maiúsculas. Com isso,
um script em Python foi usado para identificar a similaridade entre as duas listas de cidades.

'''

In [ ]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import unidecode
import math

In [ ]:
def f(str):
    return (unidecode.unidecode(str))

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
path = "/drive/My Drive/Pinpoint/energia_eolica/atos-outorgas-aneel.xlsx"
atos = pd.read_excel(path, sheet_name = 'atos-outorgas-aneel', dtype = 'str')
atos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43797 entries, 0 to 43796
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   DatGeracaoConjuntoDados  43797 non-null  object
 1   DatPublicacao            43796 non-null  object
 2   DscNumAto                43797 non-null  object
 3   DscEmpreendimento        43797 non-null  object
 4   CodCEG                   43768 non-null  object
 5   IdeNucleoCEG             43791 non-null  object
 6   SigTipoGeracao           43797 non-null  object
 7   DscAssunto               29229 non-null  object
 8   DscObjeto                43797 non-null  object
 9   DscAmbiente              43797 non-null  object
 10  DscTipoAto               43797 non-null  object
 11  DscProcesso              43793 non-null  object
 12  DscCombustivel           43797 non-null  object
 13  MdaPotenciaInstaladaMW   42789 non-null  object
 14  MdaAmpliacaoReducaoMW    42941 non-nul

In [ ]:
atos["DscCombustivel"].unique()

array(['hídrica', 'radiação solar', 'óleo diesel',
       'bagaço de cana-de-açúcar', 'licor negro', 'não se aplica',
       'carvão mineral', 'gás natural', 'calor de processo - cm',
       'óleo combustível', 'gás de refinaria', 'resíduos florestais',
       'outros energéticos de petróleo', 'gás de alto forno - cm',
       'cavaco de madeira', 'lenha', 'carvão vegetal', 'biogás - agr',
       'gás de alto forno - biomassa', 'capim elefante', 'urânio',
       'cinética do vento', 'biogás - ru', 'casca de arroz',
       'óleos vegetais', 'resíduos de madeira', 'calor de processo - of',
       'biogás - floresta', 'calor de processo - gn',
       'resíduos sólidos urbanos - ru', 'biodiesel', 'ute'], dtype=object)

In [ ]:
atos_vento = atos[(atos['DscCombustivel'] == 'cinética do vento')].copy()
atos_vento.shape

(16622, 20)

In [ ]:
atos_vento["NomMunicipio"].unique()

array(['Sento Sé', 'Aquiraz', 'São Gonçalo do Amarante',
       'São Bento do Norte', 'Morro do Chapéu', 'Barreirinhas',
       'Várzea Nova', 'Beberibe', 'Camocim', 'Amontada', 'Paracuru',
       'São Francisco de Itabapoana', 'Acaraú', 'Aracati', 'Água Doce',
       'Giruá', 'Mataraca', 'Alhandra', 'Osório', 'Itarema', 'Trairi',
       'Guanambi', 'Igaporã', 'Caetité', 'Palmares do Sul',
       'Areia Branca', 'Santana do Livramento',
       'Caiçara do Norte, São Bento do Norte', 'Viamão',
       'Caiçara do Nore', 'Parazinho', 'João Câmara', 'Pedra Grande',
       'São Miguel do Gostoso', 'Tianguá', 'Ubajara', 'Lagoa Nova',
       'Bodó', 'Parnaíba', 'Pedra Grande, São Bento do Norte',
       'Paulino Neves', 'Tutóia', 'Santa Vitória do Palmar', 'Chuí',
       'Tacaratu', 'Icapuí', 'ICAPUÍ', 'Touros', 'Caiçara do Norte',
       'Tenente Laurentino Cruz', 'Ceará-Mirim', 'Tibau', 'Serra do Mel',
       'Fortim', 'Jandaíra', 'Campo Formoso', 'Santana do Matos',
       'RIO DO FOGO', '

In [ ]:
path = "/drive/My Drive/Pinpoint/energia_eolica/atos-outorgas-aneel.xlsx"
pop_originarios = pd.read_excel(path, sheet_name = 'municípios indígenas e quilombo', dtype = 'str')
pop_originarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2195 entries, 0 to 2194
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   NM_MUN  2195 non-null   object
 1   NM_UF   2195 non-null   object
 2   CD_MUN  2195 non-null   object
dtypes: object(3)
memory usage: 51.6+ KB


In [ ]:
pop_originarios['NM_MUN'] = pop_originarios['NM_MUN'].str.strip()
pop_originarios['NM_MUN'] = pop_originarios['NM_MUN'].str.upper()
pop_originarios['NM_MUN'] = pop_originarios['NM_MUN'].apply(f)

In [ ]:
a = pop_originarios['NM_MUN'].unique()
a_tolist = a.tolist()

In [ ]:
encontrados = []

for num, row in atos_vento.iterrows():
  cidades = row['NomMunicipio']
  if cidades == '' or (isinstance(cidades, float) and  math.isnan(cidades)):
    cidades_separadas = []
  else:
    cidades_separadas = cidades.split("/")

  DatGeracaoConjuntoDados = row['DatGeracaoConjuntoDados']
  DatPublicacao = row['DatPublicacao']
  DscNumAto = row['DscNumAto']
  DscEmpreendimento = row['DscEmpreendimento']
  CodCEG = row['CodCEG']
  IdeNucleoCEG = row['IdeNucleoCEG']
  SigTipoGeracao = row['SigTipoGeracao']
  DscAssunto = row['DscAssunto']
  DscObjeto = row['DscObjeto']
  DscAmbiente = row['DscAmbiente']
  DscTipoAto = row['DscTipoAto']
  DscProcesso = row['DscProcesso']
  DscCombustivel = row['DscCombustivel']
  MdaPotenciaInstaladaMW = row['MdaPotenciaInstaladaMW']
  MdaAmpliacaoReducaoMW = row['MdaAmpliacaoReducaoMW']
  NomAgente = row['NomAgente']
  NomMunicipio = row['NomMunicipio']
  SigUF = row['SigUF']
  SigRegimeExploracao = row['SigRegimeExploracao']
  DscRio = row['DscRio']

  for i in range(len(cidades_separadas)):
    cidades_separadas[i] = cidades_separadas[i].upper().strip()
    cidades_separadas[i] = unidecode.unidecode(cidades_separadas[i])


  similar = "NÃO"
  for vez in cidades_separadas:
    if vez in a_tolist:
      similar = "SIM"

  dicionario = {'DatGeracaoConjuntoDados': DatGeracaoConjuntoDados,
                'DatPublicacao': DatPublicacao,
                'DscNumAto': DscNumAto,
                'DscEmpreendimento': DscEmpreendimento,
                'CodCEG': CodCEG,
                'IdeNucleoCEG': IdeNucleoCEG,
                'SigTipoGeracao': SigTipoGeracao,
                'DscAssunto': DscAssunto,
                'DscObjeto': DscObjeto,
                'DscAmbiente': DscAmbiente,
                'DscTipoAto': DscTipoAto,
                'DscProcesso': DscProcesso,
                'DscCombustivel': DscCombustivel,
                'MdaPotenciaInstaladaMW': MdaPotenciaInstaladaMW,
                'MdaAmpliacaoReducaoMW': MdaAmpliacaoReducaoMW,
                'NomAgente': NomAgente,
                'NomMunicipio': NomMunicipio,
                'SigUF': SigUF,
                'SigRegimeExploracao': SigRegimeExploracao,
                'DscRio': DscRio,
                'lista_municipios': cidades_separadas,
                'similar_com_povos_originarios': similar}
  encontrados.append(dicionario)

In [ ]:
len(encontrados)

16622

In [ ]:
df_encontrados = pd.DataFrame(encontrados)

In [ ]:
df_encontrados.to_excel('/drive/My Drive/Pinpoint/energia_eolica/atos-outorgas-aneel_comparados_com_indigenas_quilombolas.xlsx',sheet_name='Sheet1',index=False)